In [4]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import os
import matplotlib as plt
import numpy as np
%matplotlib inline

### 1. Carregando Tabelas
Arquivos baixados do painel de remunerações de magistrados disponibilizado pelo CNJ: https://paineis.cnj.jus.br/QvAJAXZfc/opendoc.htm?document=qvw_l%2FPainelCNJ.qvw&host=QVS%40neodimio03&anonymous=true&sheet=shPORT63Relatorios

In [ ]:
%%time

# Remuneração resumida
files = [ f for f in os.listdir('_dados_cnj/_geral/') if f != '.DS_Store']
df_folha_resumida = pd.read_excel('_dados_cnj/_geral/' + files[0])

for i in range(1,len(files)):
    df_folha_resumida = pd.concat([df_folha_resumida, pd.read_excel('_dados_cnj/_geral/' + files[i])])
    if i % 10 == 0:
        print('resumida', i, len(df_folha_resumida))

# Pessoais
files = [ f for f in os.listdir('_dados_cnj/_pessoais/') if f != '.DS_Store']
df_folha_pessoais = pd.read_excel('_dados_cnj/_pessoais/' + files[0])

for i in range(1,len(files)):
    df_folha_pessoais = pd.concat([df_folha_pessoais, pd.read_excel('_dados_cnj/_pessoais/' + files[i])])
    if i % 10 == 0:
        print('pessoais', i, len(df_folha_pessoais))
        
# Indenizações
files = [ f for f in os.listdir('_dados_cnj/_indenizacoes/') if f != '.DS_Store']
df_folha_indenizacoes = pd.read_excel('_dados_cnj/_indenizacoes/' + files[0])

for i in range(1,len(files)):
    df_folha_indenizacoes = pd.concat([df_folha_indenizacoes, pd.read_excel('_dados_cnj/_indenizacoes/' + files[i])])
    if i % 10 == 0:
        print('indenizacoes', i, len(df_folha_indenizacoes))
        
# Eventuais
files = [ f for f in os.listdir('_dados_cnj/_eventuais/') if f != '.DS_Store']
df_folha_eventuais = pd.read_excel('_dados_cnj/_eventuais/' + files[0])

for i in range(1,len(files)):
    df_folha_eventuais = pd.concat([df_folha_eventuais, pd.read_excel('_dados_cnj/_eventuais/' + files[i])])
    if i % 10 == 0:
        print('eventuais', i, len(df_folha_eventuais))

In [ ]:
df_folha_resumida_ = df_folha_resumida.copy()
df_folha_eventuais_ = df_folha_eventuais.copy()
df_folha_pessoais_ = df_folha_pessoais.copy()
df_folha_indenizacoes_ = df_folha_indenizacoes.copy()

In [5]:
df_folha_resumida_ = pd.read_pickle('df_folha_resumida_.pkl')
df_folha_eventuais_ = pd.read_pickle('df_folha_eventuais_.pkl')
df_folha_pessoais_ = pd.read_pickle('df_folha_pessoais_.pkl')
df_folha_indenizacoes_ = pd.read_pickle('df_folha_indenizacoes_.pkl')

### 2. Análise das remunerações

##### 2.1. Ajustes iniciais e definição de escopo
Crio um campo chamado "abate-teto", que é o valor absoluto do desconto (pois aparece com valor positivo ou negativo, a depender do tribunal).

Crio um campo chamado "remuneracao_bruta", que significa todos os recebimentos do mês mais diárias e remuneração de outros órgãos, descontado o "abate-teto".

Crio um campo de "remuneração liquida", que é a coluna "Total (10) + (11) + (12).

Crio um campo definido como "grupo_tribunal", com uma categorização dos tribunais.

Desconsidero remunerações inferiores a R$ 20k, pois não existe magistrado recebendo menos que isso de vencimento básico (valores menores decorrem de ajustes de folha, que não interessam a esta análise).

Desconsidero magistrados denominados "0", pois indicam que são linhas de somatório das tabelas, enviadas por engano pelo tribunal.

Filtro os registros para considerar apenas folhas de 2019 e 2020


In [6]:
print(len(df_folha_resumida_), df_folha_resumida_.columns)

884805 Index(['Tribunal', 'Magistrado', 'Mês/Ano Ref.', 'Subsídio (R$)',
       'Direitos Pessoais (1)', 'Indenizações (2)', 'Direitos Eventuais (3)',
       'Total de Rendimentos (4)', 'Previdência Pública (5) (R$)',
       'Imposto de Renda (6) (R$)', 'Descontos Diversos (7) (R$)',
       'Retenção por Teto Constitucional (8) (R$)', 'Total de Descontos (9)',
       'Rendimento Líquido (10)', 'Remuneração do órgão de origem (11) (R$)',
       'Diárias (12) (R$)', 'Total (10)+(11)+(12)'],
      dtype='object')


In [7]:
print(len(df_folha_eventuais_), df_folha_eventuais_.columns)

884805 Index(['Tribunal', 'Magistrado', 'Mês/Ano Ref.',
       'Abono constitucional de 1/3 de férias (R$)',
       'Indenização de férias (R$)', 'Antecipação de férias (R$)',
       'Gratificação natalina (R$)',
       'Antecipação de gratificação natalina (R$)', 'Substituição (R$)',
       'Gratificação por exercício cumulativo (R$)',
       'Gratificação por encargo Curso/Concurso (R$)',
       'Pagamentos retroativos (R$)', 'JETON (R$)', 'Outra (R$)', 'Detalhe',
       'Outra (R$).1', 'Detalhe.1', 'Total de Direitos Eventuais'],
      dtype='object')


In [8]:
print(len(df_folha_pessoais_), df_folha_pessoais_.columns)

884805 Index(['Tribunal', 'Magistrado', 'Mês/Ano Ref.', 'Abono de permanência (R$)',
       'Outra (R$)', 'Detalhe', 'Outra (R$).1', 'Detalhe.1',
       'Total de Direitos Pessoais'],
      dtype='object')


In [9]:
print(len(df_folha_indenizacoes_), df_folha_indenizacoes_.columns)

884805 Index(['Tribunal', 'Magistrado', 'Mês/Ano Ref.', 'Auxílio-alimentação (R$)',
       'Auxílio Pré-escolar (R$)', 'Auxílio Saúde (R$)',
       'Auxílio Natalidade (R$)', 'Auxílio Moradia (R$)',
       'Ajuda de Custo (R$)', 'Outra (R$)', 'Detalhe', 'Outra (R$).1',
       'Detalhe.1', 'Outra (R$).2', 'Detalhe.2', 'Total Indenizações'],
      dtype='object')


In [10]:
def grupo_tribunal(x):
    if x == 'CNJ':
        y = 'CNJ'
    elif x in ['STJ', 'TSE', 'STM', 'TST']:
        y = 'Superiores (STJ, TSE, STM e TST)'
    elif x in ['TJMMG', 'TJMSP', 'TJMRS']:
        y = 'Justiça Militar (TJMMG, TJMSP, TJMRS)'
    elif x[:3] == 'TRF':
        y = 'Justiça Federal (TRFs)'
    elif x[:3] == 'TRT':
        y = 'Justiça do Trabalho (TRTs)'
    else:
        y = 'Justiça Estadual (TJs)'
    return y

for i in [df_folha_resumida_, df_folha_pessoais_, df_folha_indenizacoes_, df_folha_eventuais_]:
    i['grupo_tribunal'] = i['Tribunal'].map(grupo_tribunal)

In [11]:
for i in [df_folha_resumida_, df_folha_pessoais_, df_folha_indenizacoes_, df_folha_eventuais_]:
    i['ano'] = i['Mês/Ano Ref.'].apply(lambda x: x[-4:])

In [12]:
df_folha_resumida_['abate_teto'] = df_folha_resumida_['Retenção por Teto Constitucional (8) (R$)'].apply(lambda x: np.abs(x))

In [13]:
df_folha_resumida_['remuneracao_bruta'] = df_folha_resumida_['Total de Rendimentos (4)'] + df_folha_resumida_['Remuneração do órgão de origem (11) (R$)'] + df_folha_resumida_['Diárias (12) (R$)'] - df_folha_resumida_['abate_teto']
df_folha_resumida_['remuneracao_liquida'] = df_folha_resumida_['Total (10)+(11)+(12)']


In [14]:
df_folha_resumida_ = df_folha_resumida_[(df_folha_resumida_['Magistrado'] != '0') & (df_folha_resumida_['Total de Rendimentos (4)'] > 20000)]


In [15]:
df_folha_resumida_ = df_folha_resumida_[(df_folha_resumida_['ano'] == '2020') | (df_folha_resumida_['ano'] == '2019')]

In [16]:
len(df_folha_resumida_)

474671

##### 2.2. Média anual por grupo de tribunais

In [17]:
df_folha_resumida_grupo_tribunal = df_folha_resumida_[['grupo_tribunal', 'ano', 'remuneracao_liquida', 'remuneracao_bruta']].pivot_table(index = ['grupo_tribunal'], columns = ['ano']).reset_index()


In [18]:
list(df_folha_resumida_grupo_tribunal)

[('grupo_tribunal', ''),
 ('remuneracao_bruta', '2019'),
 ('remuneracao_bruta', '2020'),
 ('remuneracao_liquida', '2019'),
 ('remuneracao_liquida', '2020')]

In [19]:
df_folha_resumida_grupo_tribunal.columns = ['Grupo de Tribunais', 'Remuneração Bruta Mensal Média em 2019', 'Remuneração Bruta Mensal Média em 2020', 'Remuneração Líquida Mensal Média em 2019', 'Remuneração Líquida Mensal Média em 2020']


In [20]:
df_folha_resumida_grupo_tribunal['Variação % entre 2019 e 2020 - Líquida'] = df_folha_resumida_grupo_tribunal['Remuneração Líquida Mensal Média em 2020']/df_folha_resumida_grupo_tribunal['Remuneração Líquida Mensal Média em 2019']-1
df_folha_resumida_grupo_tribunal['Variação % entre 2019 e 2020 - Bruta'] = df_folha_resumida_grupo_tribunal['Remuneração Bruta Mensal Média em 2020']/df_folha_resumida_grupo_tribunal['Remuneração Bruta Mensal Média em 2019']-1


In [21]:
df_folha_resumida_grupo_tribunal[['Grupo de Tribunais', 'Remuneração Bruta Mensal Média em 2019', 'Remuneração Bruta Mensal Média em 2020', 'Variação % entre 2019 e 2020 - Bruta']]


,Grupo de Tribunais,Remuneração Bruta Mensal Média em 2019,Remuneração Bruta Mensal Média em 2020,Variação % entre 2019 e 2020 - Bruta
0,CNJ,"62,634.50","55,686.22",-0.11
1,Justiça Estadual (TJs),"51,037.19","49,669.53",-0.03
2,Justiça Federal (TRFs),"45,258.97","51,514.31",0.14
3,"Justiça Militar (TJMMG, TJMSP, TJMRS)","51,856.31","59,962.80",0.16
4,Justiça do Trabalho (TRTs),"44,050.97","41,793.97",-0.05
5,"Superiores (STJ, TSE, STM e TST)","45,022.17","46,218.35",0.03


In [22]:
df_folha_resumida_grupo_tribunal[['Grupo de Tribunais', 'Remuneração Líquida Mensal Média em 2019', 'Remuneração Líquida Mensal Média em 2020', 'Variação % entre 2019 e 2020 - Líquida']]


,Grupo de Tribunais,Remuneração Líquida Mensal Média em 2019,Remuneração Líquida Mensal Média em 2020,Variação % entre 2019 e 2020 - Líquida
0,CNJ,"56,475.30","48,210.24",-0.15
1,Justiça Estadual (TJs),"37,156.26","35,666.76",-0.04
2,Justiça Federal (TRFs),"30,344.39","34,348.54",0.13
3,"Justiça Militar (TJMMG, TJMSP, TJMRS)","37,157.92","44,454.41",0.20
4,Justiça do Trabalho (TRTs),"31,658.58","28,477.83",-0.10
5,"Superiores (STJ, TSE, STM e TST)","32,630.46","31,820.10",-0.02


In [23]:
df_folha_resumida_[df_folha_resumida_['ano'] == '2019']['remuneracao_liquida'].mean()

35156.466708749664

In [24]:
df_folha_resumida_[df_folha_resumida_['ano'] == '2020']['remuneracao_liquida'].mean()

34000.443146309735

In [25]:
df_folha_resumida_[df_folha_resumida_['ano'] == '2019']['remuneracao_bruta'].mean()

48772.629521753064

In [26]:
df_folha_resumida_[df_folha_resumida_['ano'] == '2020']['remuneracao_bruta'].mean()

48215.3999892845

##### 2.3. Variação da remuneração média anual, entre 2019 e 2020, por magistrado

In [27]:
df_folha_resumida_pivot = df_folha_resumida_[['Magistrado', 'Tribunal', 'grupo_tribunal', 'ano', 'remuneracao_bruta', 'remuneracao_liquida']].pivot_table(index = ['Magistrado', 'Tribunal', 'grupo_tribunal'], columns = ['ano']).reset_index()


In [28]:
list(df_folha_resumida_pivot)

[('Magistrado', ''),
 ('Tribunal', ''),
 ('grupo_tribunal', ''),
 ('remuneracao_bruta', '2019'),
 ('remuneracao_bruta', '2020'),
 ('remuneracao_liquida', '2019'),
 ('remuneracao_liquida', '2020')]

In [30]:
df_folha_resumida_pivot = df_folha_resumida_[['Magistrado', 'Tribunal', 'grupo_tribunal', 'ano', 'remuneracao_bruta', 'remuneracao_liquida']].pivot_table(index = ['Magistrado', 'Tribunal', 'grupo_tribunal'], columns = ['ano']).reset_index()
df_folha_resumida_pivot.columns = ['Magistrado', 'Tribunal', 'grupo_tribunal', '2019_bruta', '2020_bruta', '2019_liquida', '2020_liquida']
df_folha_resumida_pivot['var_19_20_bruta'] = (df_folha_resumida_pivot['2020_bruta']/df_folha_resumida_pivot['2019_bruta'])-1
df_folha_resumida_pivot['var_19_20_liquida'] = (df_folha_resumida_pivot['2020_liquida']/df_folha_resumida_pivot['2019_liquida'])-1


In [31]:
df_folha_resumida_pivot[['grupo_tribunal', 'var_19_20_bruta', 'var_19_20_liquida']].pivot_table(index = ['grupo_tribunal'])

,var_19_20_bruta,var_19_20_liquida
grupo_tribunal,,
CNJ,-0.09,-0.08
Justiça Estadual (TJs),0.04,0.03
Justiça Federal (TRFs),0.16,0.15
"Justiça Militar (TJMMG, TJMSP, TJMRS)",0.09,0.11
Justiça do Trabalho (TRTs),-0.04,-0.08
"Superiores (STJ, TSE, STM e TST)",0.03,-0.02


In [32]:
df_folha_resumida_pivot['var_19_20_bruta'].mean()

0.03414153803287967

In [33]:
df_folha_resumida_pivot['var_19_20_liquida'].mean()

0.020732219155861334

##### 2.4. Cumprimento do teto salarial

In [34]:
teto = 39293
df_folha_resumida_['teto'] = teto

def vl_maior_teto(row):
    if row['remuneracao_bruta'] > row['teto']:
        y = row['remuneracao_bruta'] - row['teto']
    else:
        y = 0
    return y

df_folha_resumida_['vl_maior_teto'] = df_folha_resumida_.apply(vl_maior_teto, axis = 1)
df_folha_resumida_['dummy_maior_teto'] = df_folha_resumida_['vl_maior_teto'].apply(lambda x: x > 0)
df_folha_resumida_['perc_vl_maior_teto'] = df_folha_resumida_['vl_maior_teto']/df_folha_resumida_['teto']
df_folha_resumida_[['Tribunal', 'remuneracao_bruta', 'vl_maior_teto', 'dummy_maior_teto', 'perc_vl_maior_teto']].sample(10)


,Tribunal,remuneracao_bruta,vl_maior_teto,dummy_maior_teto,perc_vl_maior_teto
2690,TJPE,"45,986.81","6,693.81",True,0.17
10639,TRT15,"56,163.92","16,870.92",True,0.43
104586,TJSP,"100,540.70","61,247.70",True,1.56
19333,TRT3,"38,304.99",0.00,False,0.00
1022,TRT11,"33,689.11",0.00,False,0.00
8543,TRF5,"40,633.40","1,340.40",True,0.03
1421,TRT3,"34,599.19",0.00,False,0.00
4690,TRT6,"34,599.19",0.00,False,0.00
33014,TJPR,"52,783.29","13,490.29",True,0.34
116701,TJSP,"57,108.50","17,815.50",True,0.45


In [35]:
df_folha_resumida_['perc_vl_maior_teto'].describe()

count   474,671.00
mean          0.28
std           0.50
min           0.00
25%           0.00
50%           0.05
75%           0.38
max          32.05
Name: perc_vl_maior_teto, dtype: float64

In [36]:
len(df_folha_resumida_[df_folha_resumida_['dummy_maior_teto'] == True])

298194

In [37]:
len(df_folha_resumida_[df_folha_resumida_['dummy_maior_teto'] == True])/len(df_folha_resumida_)

0.6282119615481039

In [38]:
df_folha_resumida_teto = df_folha_resumida_[['grupo_tribunal', 'Magistrado', 'dummy_maior_teto']].groupby(['grupo_tribunal']).agg({'dummy_maior_teto': 'sum', 'Magistrado': 'count'}).reset_index()
df_folha_resumida_teto.columns = ['Grupo de Tribunais', 'Número de Folhas Acima do Teto', 'Número de Folhas']
df_folha_resumida_teto['Percentual de Folhas Acima do Teto'] = df_folha_resumida_teto['Número de Folhas Acima do Teto']/df_folha_resumida_teto['Número de Folhas']
df_folha_resumida_teto


,Grupo de Tribunais,Número de Folhas Acima do Teto,Número de Folhas,Percentual de Folhas Acima do Teto
0,CNJ,83,91,0.91
1,Justiça Estadual (TJs),212163,314034,0.68
2,Justiça Federal (TRFs),39405,49251,0.80
3,"Justiça Militar (TJMMG, TJMSP, TJMRS)",826,1036,0.80
4,Justiça do Trabalho (TRTs),43615,106689,0.41
5,"Superiores (STJ, TSE, STM e TST)",2102,3570,0.59


##### 2.5. Principais penduricalhos em 2020

In [50]:
df_penduricalhos = pd.concat([pd.melt(df_folha_pessoais_[df_folha_pessoais_['ano'] == '2020'], id_vars = ['Tribunal', 'Magistrado', 'Mês/Ano Ref.'], value_vars = ['Abono de permanência (R$)']),
                             pd.melt(df_folha_eventuais_[df_folha_eventuais_['ano'] == '2020'], id_vars = ['Tribunal', 'Magistrado', 'Mês/Ano Ref.'], value_vars = ['Abono constitucional de 1/3 de férias (R$)', 'Indenização de férias (R$)', 'Antecipação de férias (R$)', 'Gratificação natalina (R$)', 'Antecipação de gratificação natalina (R$)', 'Substituição (R$)', 'Gratificação por exercício cumulativo (R$)', 'Gratificação por encargo Curso/Concurso (R$)', 'Pagamentos retroativos (R$)', 'JETON (R$)']),
                             pd.melt(df_folha_indenizacoes_[df_folha_indenizacoes_['ano'] == '2020'], id_vars = ['Tribunal', 'Magistrado', 'Mês/Ano Ref.'], value_vars = ['Auxílio-alimentação (R$)', 'Auxílio Pré-escolar (R$)', 'Auxílio Saúde (R$)', 'Auxílio Natalidade (R$)', 'Auxílio Moradia (R$)', 'Ajuda de Custo (R$)'])
                            ])

df_penduricalhos = df_penduricalhos.rename(columns = {'variable': 'Tipo_Acrescimo_Salarial', 'value': 'Valor'}).reset_index(drop = True)


In [40]:
# Corrigindo valores registrados como texto
def corrige_valor(x):
    if x == '-':
        y = 0
    else:
        y = float(str(x).replace(',','.'))
    return y

df_penduricalhos['Valor'] = df_penduricalhos['Valor'].map(corrige_valor)

In [41]:
len(df_penduricalhos)

3937863

In [42]:
set(df_penduricalhos['Tipo_Acrescimo_Salarial'])

{'Abono constitucional de 1/3 de férias (R$)',
 'Abono de permanência (R$)',
 'Ajuda de Custo (R$)',
 'Antecipação de férias (R$)',
 'Antecipação de gratificação natalina (R$)',
 'Auxílio Moradia (R$)',
 'Auxílio Natalidade (R$)',
 'Auxílio Pré-escolar (R$)',
 'Auxílio Saúde (R$)',
 'Auxílio-alimentação (R$)',
 'Gratificação natalina (R$)',
 'Gratificação por encargo Curso/Concurso (R$)',
 'Gratificação por exercício cumulativo (R$)',
 'Indenização de férias (R$)',
 'JETON (R$)',
 'Pagamentos retroativos (R$)',
 'Substituição (R$)'}

In [43]:
df_penduricalhos[['Tipo_Acrescimo_Salarial', 'Valor']].groupby(['Tipo_Acrescimo_Salarial']).sum().reset_index().sort_values('Valor', ascending = False)


,Tipo_Acrescimo_Salarial,Valor
10,Gratificação natalina (R$),"566,834,658.45"
15,Pagamentos retroativos (R$),"564,570,501.50"
13,Indenização de férias (R$),"448,093,993.29"
12,Gratificação por exercício cumulativo (R$),"375,929,749.34"
0,Abono constitucional de 1/3 de férias (R$),"230,140,542.38"
1,Abono de permanência (R$),"197,530,030.85"
9,Auxílio-alimentação (R$),"181,669,608.94"
4,Antecipação de gratificação natalina (R$),"170,812,443.74"
8,Auxílio Saúde (R$),"143,131,339.77"
16,Substituição (R$),"54,313,940.59"


In [44]:
df_penduricalhos[df_penduricalhos['Tribunal'] == 'CNJ'][['Tipo_Acrescimo_Salarial', 'Valor']].groupby(['Tipo_Acrescimo_Salarial']).sum().reset_index().sort_values('Valor', ascending = False)


,Tipo_Acrescimo_Salarial,Valor
2,Ajuda de Custo (R$),"485,961.92"
8,Auxílio Saúde (R$),"209,292.14"
5,Auxílio Moradia (R$),"126,910.91"
4,Antecipação de gratificação natalina (R$),"115,234.97"
9,Auxílio-alimentação (R$),"50,054.40"
15,Pagamentos retroativos (R$),"14,953.01"
0,Abono constitucional de 1/3 de férias (R$),"3,048.48"
11,Gratificação por encargo Curso/Concurso (R$),286.69
7,Auxílio Pré-escolar (R$),0.00
1,Abono de permanência (R$),0.00


##### 2.6. Por que o aumento tao grande nos militares e TRFs?

In [45]:
df_penduricalhos_19_20 = pd.concat([pd.melt(df_folha_pessoais_[(df_folha_pessoais_['ano'] == '2020') | (df_folha_pessoais_['ano'] == '2019')], id_vars = ['Tribunal', 'Magistrado', 'Mês/Ano Ref.', 'ano'], value_vars = ['Abono de permanência (R$)']),
                             pd.melt(df_folha_eventuais_[(df_folha_eventuais_['ano'] == '2020') | (df_folha_eventuais_['ano'] == '2019')], id_vars = ['Tribunal', 'Magistrado', 'Mês/Ano Ref.', 'ano'], value_vars = ['Abono constitucional de 1/3 de férias (R$)', 'Indenização de férias (R$)', 'Antecipação de férias (R$)', 'Gratificação natalina (R$)', 'Antecipação de gratificação natalina (R$)', 'Substituição (R$)', 'Gratificação por exercício cumulativo (R$)', 'Gratificação por encargo Curso/Concurso (R$)', 'Pagamentos retroativos (R$)', 'JETON (R$)']),
                             pd.melt(df_folha_indenizacoes_[(df_folha_indenizacoes_['ano'] == '2020') | (df_folha_indenizacoes_['ano'] == '2019')], id_vars = ['Tribunal', 'Magistrado', 'Mês/Ano Ref.', 'ano'], value_vars = ['Auxílio-alimentação (R$)', 'Auxílio Pré-escolar (R$)', 'Auxílio Saúde (R$)', 'Auxílio Natalidade (R$)', 'Auxílio Moradia (R$)', 'Ajuda de Custo (R$)'])
                            ])

df_penduricalhos_19_20 = df_penduricalhos_19_20.rename(columns = {'variable': 'Tipo_Acrescimo_Salarial', 'value': 'Valor'}).reset_index(drop = True)


In [46]:
df_penduricalhos_19_20['Valor'] = df_penduricalhos_19_20['Valor'].map(corrige_valor)


In [47]:
df_penduricalhos_19_20['grupo_tribunal'] = df_penduricalhos_19_20['Tribunal'].map(grupo_tribunal)

In [48]:
df_variacao_penduricalhos = df_penduricalhos_19_20[(df_penduricalhos_19_20['grupo_tribunal'] == 'Justiça Militar (TJMMG, TJMSP, TJMRS)') | (df_penduricalhos_19_20['grupo_tribunal'] == 'Justiça Federal (TRFs)')][['Tipo_Acrescimo_Salarial', 'ano', 'Valor']].pivot_table(index = ['Tipo_Acrescimo_Salarial'], columns = ['ano'])
df_variacao_penduricalhos['Diferença entre 2019 e 2020'] = df_variacao_penduricalhos['Valor', '2020'] - df_variacao_penduricalhos['Valor', '2019']
df_variacao_penduricalhos.sort_values('Diferença entre 2019 e 2020', ascending = False)

Valor           \
ano                                              2019     2020   
Tipo_Acrescimo_Salarial                                          
Pagamentos retroativos (R$)                    142.26 5,208.24   
Gratificação natalina (R$)                   2,111.77 2,827.26   
Gratificação por exercício cumulativo (R$)   6,416.71 7,102.52   
Indenização de férias (R$)                     432.00   776.55   
Abono constitucional de 1/3 de férias (R$)   1,327.82 1,600.04   
Abono de permanência (R$)                      429.41   620.83   
Antecipação de férias (R$)                     386.46   417.18   
JETON (R$)                                       0.00     0.00   
Auxílio Moradia (R$)                             0.11     0.01   
Auxílio Natalidade (R$)                          1.50     1.32   
Auxílio Pré-escolar (R$)                       164.89   164.03   
Auxílio-alimentação (R$)                       787.87   784.46   
Auxílio Saúde (R$)                             168.67   159.80   
Ajuda de Custo (R$)                            109.03    80.41   
Gratificação por encargo Curso/Concurso (R$)    46.80    16.80   
Substituição (R$)                              230.80   195.71   
Antecipação de gratificação natalina (R$)    1,311.84   754.39   

                                             Diferença entre 2019 e 2020  
ano                                                                       
Tipo_Acrescimo_Salarial                                                   
Pagamentos retroativos (R$)                                     5,065.99  
Gratificação natalina (R$)                                        715.49  
Gratificação por exercício cumulativo (R$)                        685.80  
Indenização de férias (R$)                                        344.55  
Abono constitucional de 1/3 de férias (R$)                        272.21  
Abono de permanência (R$)                                         191.42  
Antecipação de férias (R$)                                         30.72  
JETON (R$)                                                          0.00  
Auxílio Moradia (R$)                                               -0.10  
Auxílio Natalidade (R$)                                            -0.18  
Auxílio Pré-escolar (R$)                                           -0.86  
Auxílio-alimentação (R$)                                           -3.41  
Auxílio Saúde (R$)                                                 -8.87  
Ajuda de Custo (R$)                                               -28.62  
Gratificação por encargo Curso/Concurso (R$)                      -30.00  
Substituição (R$)                                                 -35.09  
Antecipação de gratificação natalina (R$)                        -557.45

##### 2.8. Grandes Números

In [49]:
print('Número de Magistrados, incluindo aposentados, com folha analisada, de 2019 a 2020: ' + str(len(set(df_folha_resumida_['Magistrado']))))
print('Número de Magistrados, incluindo aposentados, com folha analisada, em 2020: ' + str(len(set(df_folha_resumida_[df_folha_resumida_['ano'] == '2020']['Magistrado']))))
print('Número de folhas analisadas, de 2019 a 2020: ' + str(len(df_folha_resumida_)))
print('Número de folhas analisadas, em 2020: ' + str(len(df_folha_resumida_[df_folha_resumida_['ano'] == '2020'])))


Número de Magistrados, incluindo aposentados, com folha analisada, de 2019 a 2020: 24906
Número de Magistrados, incluindo aposentados, com folha analisada, em 2020: 21602
Número de folhas analisadas, de 2019 a 2020: 474671
Número de folhas analisadas, em 2020: 226925
